In [ ]:
from pinecone import Pinecone,ServerlessSpec
from openai import OpenAI
import pandas as pd 
import os 
import dotenv
import time
dotenv.load_dotenv()

True

In [17]:
token = os.getenv('RUNPOD_TOKEN')
open_ai_base_url = os.getenv('RUNPOD_EMBEDDING_URL')
model_name = os.getenv('MODEL_NAME')
pinecone_api_key= os.getenv('PINECONE_API_KEY')
index_name = os.getenv('PINECONE_INDEX_NAME')

In [18]:
pc = Pinecone(api_key=pinecone_api_key)
client  = OpenAI(
    api_key=token,
    base_url=open_ai_base_url
)

In [19]:
output = client.embeddings.create(input=["Hello World!"],model=model_name)
embedding = output.data[0].embedding
print(embedding)

[-0.0034090846311300993, -0.011942288838326931, 0.0417560413479805, -0.03821532428264618, 0.02428138256072998, 0.013712647370994091, 0.010973170399665833, 0.0481049120426178, 0.02124430239200592, 0.0142391761764884, -0.0030962196178734303, -0.03845951333642006, 0.00943173747509718, 0.04923427850008011, 0.038154277950525284, 0.022419452667236328, 0.035346124321222305, -0.013048762455582619, -0.1088465228676796, -0.020649094134569168, 0.10616046190261841, 0.04743339866399765, -0.02048121578991413, -0.04987527057528496, 0.0010606888681650162, -0.015276972204446793, -0.014643611386418343, 0.035498738288879395, -0.001640633912757039, -0.17605909705162048, -0.013804217800498009, -0.011927027255296707, 0.09682029485702515, -0.010431379079818726, 0.03757433220744133, -0.018237745389342308, -0.0025963985826820135, 0.047769155353307724, -0.04456419497728348, 0.008119230158627033, 0.041939180344343185, -0.012537495233118534, -0.0015004170127213001, -0.009607247076928616, 0.014063666574656963, -0.

In [20]:
len(embedding)

384

# Wrangle Dataset

In [21]:
df = pd.read_json(r"products\products.jsonl", lines=True)
df.head()

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp
2,Latte,Coffee,"Smooth and creamy, our latte combines rich esp...","[Espresso, Steamed Milk, Milk Foam]",4.75,4.8,Latte.jpg
3,Chocolate Chip Biscotti,Bakery,"Crunchy and delightful, this chocolate chip bi...","[Flour, Sugar, Chocolate Chips, Eggs, Almonds,...",2.50,4.6,chocolat_biscotti.jpg
4,Espresso shot,Coffee,"A bold shot of rich espresso, our espresso is ...",[Espresso],2.00,4.9,Espresso_shot.webp


In [22]:
df['text'] = df['name']+" : "+df['description']+\
" -- ingredients: "+df['ingredients'].astype(str)+\
" -- price: "+df['price'].astype(str)+\
" -- rating: "+df['rating'].astype(str)

In [23]:
df['text'].head(1)

0    Cappuccino : A rich and creamy cappuccino made...
Name: text, dtype: object

In [24]:
texts = df['text'].to_list()

In [25]:
with open(r"products\Merry's_way_about_us.txt") as f:
    Marry_way_about_section = f.read()

Marry_way_about_section = "Coffee shop Merry's Way about section : " + Marry_way_about_section
texts.append(Marry_way_about_section)

In [26]:
Marry_way_about_section

"Coffee shop Merry's Way about section : Welcome to Merry's Way Coffee, your neighborhood coffee shop located in the heart of Greenwich Village, New York City. At Merry's Way, we believe that coffee is more than just a drinkâ€”itâ€™s an experience, a moment of joy, and a way to connect with others.\n\nOur Story\nFounded in 2015, Merryâ€™s Way started as a small family-owned cafÃ© with one mission: to share the love of quality, ethically-sourced coffee with our community.\n\nMerry's passion for travel and coffee led her on a journey across South America, where she handpicked partnerships with small farms and cooperatives. We ensure that every cup we brew tells a story of dedication and care, from farm to table. Our beans are roasted in-house to bring out unique flavors that reflect the regions where they were grown.\n\nDelivery & Locations Served\nIn addition to offering a cozy place to enjoy coffee in our cafÃ©, we proudly deliver to Greenwich Village, SoHo, West Village, and Lower Man

In [27]:
with open(r"products\menu_items_text.txt") as f:
    menu_items_text = f.read()

menu_items_text = "Menu Items : " + menu_items_text
texts.append(menu_items_text)

In [28]:
menu_items_text

'Menu Items : Menu Items\n\nCappuccino - $4.50\nJumbo Savory Scone - $3.25\nLatte - $4.75\nChocolate Chip Biscotti - $2.50\nEspresso shot - $2.00\nHazelnut Biscotti - $2.75\nChocolate Croissant - $3.75\nDark chocolate (Drinking Chocolate) - $5.00\nCranberry Scone - $3.50\nCroissant - $3.25\nAlmond Croissant - $4.00\nGinger Biscotti - $2.50\nOatmeal Scone - $3.25\nGinger Scone - $3.50\nChocolate syrup - $1.50\nHazelnut syrup - $1.50\nCarmel syrup - $1.50\nSugar Free Vanilla syrup - $1.50\nDark chocolate (Packaged Chocolate) - $3.00'

# Generate Embeddings

In [29]:
output = client.embeddings.create(input=texts,model=model_name)

In [30]:
embeddings = output.data
embeddings[0]

Embedding(embedding=[-0.014078277163207531, -0.06446248292922974, 0.003805713029578328, 0.004784325137734413, 0.030125223100185394, -0.045630402863025665, 0.03723685070872307, 0.010690334253013134, -0.027744507417082787, -0.05091071128845215, -0.052436813712120056, -0.03366577625274658, -0.0004516303597483784, -0.043768562376499176, 0.03757259249687195, -0.01718389242887497, -0.019488302990794182, -0.08063914626836777, -0.07587771862745285, -0.007950983941555023, 0.0483773872256279, -0.06122715026140213, -0.10749851912260056, -0.03699267655611038, 0.03549709916114807, -0.04856051877140999, 0.08790338784456253, -0.01983930729329586, 0.001731170224957168, -0.13075628876686096, -0.012933701276779175, 0.009805195964872837, -0.02801920473575592, -0.025821620598435402, -0.0350392684340477, 0.0010091339936479926, 0.06751468777656555, -0.07593876123428345, 0.047095462679862976, 0.0005737183964811265, -0.002592463279142976, 0.029804741963744164, -0.06415726244449615, -0.002075496595352888, -0.0

# Push Data to Vector Database (Pinecone)

In [31]:
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [34]:
#wait for index to be ready
while not pc.describe_index(index_name).status.ready:
    time.sleep(1)
index = pc.Index(index_name)
vectors=[]
for text, e in zip(texts,embeddings):
    entry_id = text.split(':')[0]
    vectors.append({
        "id": entry_id,
        "values": e.embedding,
        "metadata": {"text":text}
    }
    )

index.upsert(vectors=vectors,namespace='ns1')

{'upserted_count': 20}

# Get Closest documents

In [35]:
output = client.embeddings.create(input=["Is Cappuccino lactose free?"],model=model_name)
embedding = output.data[0].embedding

In [36]:
results = index.query(
    namespace='ns1',
    vector=embedding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

In [37]:
results

{'matches': [{'id': 'Cappuccino ',
              'metadata': {'text': 'Cappuccino : A rich and creamy cappuccino '
                                   'made with freshly brewed espresso, steamed '
                                   'milk, and a frothy milk cap. This '
                                   'delightful drink offers a perfect balance '
                                   'of bold coffee flavor and smooth milk, '
                                   'making it an ideal companion for relaxing '
                                   'mornings or lively conversations. -- '
                                   "ingredients: ['Espresso', 'Steamed Milk', "
                                   "'Milk Foam'] -- price: 4.5 -- rating: 4.7"},
              'score': 0.729054034,
              'values': []},
             {'id': 'Sugar Free Vanilla syrup ',
              'metadata': {'text': 'Sugar Free Vanilla syrup : Enjoy the sweet '
                                   'flavor of vanilla without th